# 사용자기반 - SGD코드 초안

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings(action='ignore')
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

C:\Users\PC\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [9]:
upper1=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\user_based_반팔left.xlsx')
upper2=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\user_based_홍균.xlsx')
upper3=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\사용자기반.하의(2).xlsx')
upper4=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\사용자기반하의왼쪽.xlsx')

df=pd.concat([upper1,upper2,upper3,upper4]).reset_index()
df.review=df.review.str.replace('\n','')
df=df.replace('Null',np.nan)
df.reset_index(inplace=True)
df.drop(['level_0','index'],axis=1,inplace=True)

df=df.drop_duplicates(['review'])
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)

In [10]:
#키 추가
cm=[]
for i in range(df.shape[0]):
    if re.findall(r'\d+cm',str(df['user_info'].iloc[i])) !=[]:
        cm.append(re.findall(r'\d+cm',df['user_info'].iloc[i]))
    else:
        cm.append(['Null'])
df=df.copy()        
df['cm']=cm

#몸무게추가
kg=[]
for i in range(df.shape[0]):
    if re.findall(r'\d+kg',str(df['user_info'].iloc[i])) !=[]:
        kg.append(re.findall(r'\d+kg',df['user_info'].iloc[i]))
    else:
        kg.append(['Null'])
df=df.copy()        
df['kg']=kg

#size추가
user_size=[]
for i in range(df.shape[0]):
    if re.findall(r'[A-Z]+',str(df['user_info'].iloc[i])) !=[]:
        user_size.append(re.findall(r'[A-Z]+',df['user_info'].iloc[i]))
    else:
        user_size.append(['Null'])
df=df.copy()        
df['user_size']=user_size

#리스트에 담겨 있어서 꺼내기
c=[];d=[];e=[]
for i in range(df.shape[0]):
    c.append(df['cm'][i][0])
    d.append(df['kg'][i][0])
    e.append(df['user_size'][i][0])
    
df['cm']=c
df.cm=df.cm.str.replace('cm','')
df['kg']=d
df.kg=df.kg.str.replace('kg','')
df['user_size']=e


In [11]:
df=df.replace('Null',np.nan)
df = df.astype({'cm': 'float'})
df = df.astype({'kg': 'float'})
df['bmi']=df.kg/(df.cm*df.cm)*10000

In [13]:
user_item_rating = df.pivot_table("score",index = "user_id",columns = "item_name")
user_item_rating.shape

(72031, 1747)

In [111]:
user_item_rating.isnull().sum().sum()

125746473

In [14]:
# rmse
from sklearn.metrics import mean_squared_error
def get_rmse(R, P, Q, non_zeros):
    error= 0
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [15]:
def matrix_factorization(R,K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # p 와 q 매트릭스의 크기를 지정, 정규 분포를 가진 랜덤한 값 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size = (num_users,K))
    Q = np.random.normal(scale=1./K, size = (num_items,K))
    
    prev_rmse=10000
    break_count=0
    
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros = [ ( i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
    
    # SGD 기법으로 P와 Q 매트릭스를 업데이트
    for step in range(steps):
        for i,j,r in non_zeros:
            # 실제 값 예측 값 차이인 오류
            eij = r - np.dot(P[i,:], Q[j,:].T)
            
            # 정규화를 반영한 SGD 업데이트 공식적용
            P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda * P[i,:])
            Q[j,:] = Q[j,:] + learning_rate * (eij * P[i,:] - r_lambda * Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step %10) == 0 :
            print('### iteration step: ', step, 'rmse:', rmse)
            
    return P,Q

In [16]:
P, Q = matrix_factorization(user_item_rating.values, K=50, steps=200, learning_rate=0.01,
                           r_lambda=0.01)

### iteration step:  0 rmse: 9.537529172502493
### iteration step:  10 rmse: 0.9938496369087061
### iteration step:  20 rmse: 0.5090422268079211
### iteration step:  30 rmse: 0.2992563606285449
### iteration step:  40 rmse: 0.12840616531013116
### iteration step:  50 rmse: 0.06118623403832383
### iteration step:  60 rmse: 0.0410318283429276
### iteration step:  70 rmse: 0.034480851262465084
### iteration step:  80 rmse: 0.03235069167270967
### iteration step:  90 rmse: 0.031444849605054725
### iteration step:  100 rmse: 0.03096989886687348
### iteration step:  110 rmse: 0.0306228500690071
### iteration step:  120 rmse: 0.030293955090808053
### iteration step:  130 rmse: 0.02995815820865567
### iteration step:  140 rmse: 0.029614488841155872
### iteration step:  150 rmse: 0.02926641062629963
### iteration step:  160 rmse: 0.02891734698489346
### iteration step:  170 rmse: 0.028569986545125735
### iteration step:  180 rmse: 0.028226332412519595
### iteration step:  190 rmse: 0.0278878480

In [17]:
pred_matrix = np.dot(P, Q.T)

In [18]:
user_item_pred_matrix = pd.DataFrame(data= pred_matrix, index= user_item_rating.index,
                                  columns = user_item_rating.columns)
user_item_pred_matrix.head(3)

item_name,#0085 greysh blue crop jeans,#0186 Heritage black mannish crop fit,#0198 Sensitive -A standard fit,#0199 Sensitive -B standard fit,#0203 off-white straight fit,#0214 Moderation indigo straight fit,#0215 Melt black straight fit,#0220 Elaboration straight crop fit,#0223 Melt black straight crop fit,#0245 Black crude wide fit,...,헤비웨이트 오버사이즈 후디드 스웨트셔츠 [미디엄 그레이],헤비코튼 자수로고 나시 3 color,화란 세미오버 니트 라벤더,화란 세미오버 니트 블랙,화이트 중기장 레이어드 슬리브리스 티셔츠 (421142LY61),화이트라벨 오버롤 네이비,후디드 스웨트셔츠 [멜란지 그레이],휩코드 세미 와이드 히든 밴딩 슬랙스 [블랙],휩코드 와이드 히든 밴딩 슬랙스 [블랙],히든 버튼 드레스 셔츠 [화이트]
user_id,,,,,,,,,,,,,,,,,,,,,
LV 1 Aqym,-2.454384,-0.414883,-0.531522,0.884831,-1.224554,-0.302517,-3.652410,-0.989883,-1.511616,-0.215520,...,1.691563,-0.596890,0.357980,-2.225717,1.457064,-1.737743,0.526686,0.552760,-0.617948,-0.294693
LV 1 Ehrlich,-0.427638,-0.514204,0.269507,0.346499,-1.194945,0.092153,1.599933,-0.854861,0.751435,-0.806503,...,-0.334104,-0.656417,1.002124,0.233827,-1.673413,1.079360,1.196556,-0.271517,1.374902,-0.175463
LV 1 Yang30,-0.549211,2.672895,-0.043729,-1.114513,-0.464872,-2.680987,-1.202512,-1.647771,0.637758,-1.946998,...,-0.230776,-0.495502,1.404265,-1.669899,-1.138639,1.158510,-1.164455,-1.387552,2.457871,-2.070284


In [19]:
pd.DataFrame(user_item_pred_matrix.iloc[0].sort_values(ascending=False))

,LV 1 Aqym
item_name,
슬릿 넥 피케 폴로 셔츠 [블랙],9.964431
[SET] 클럽 팀 클래식 셋업,4.444396
Arch Logo Hoodie Navy,3.677835
RMTCRW LOGO SWEATSHIRT_BLACK,3.654900
[일주일만이가격] 커츠 B30 워시드,3.615261
...,...
RAINBOW REFLECTIVE LOGO TEE BLACK(MG2BSMT504A),-3.648204
#0215 Melt black straight fit,-3.652410
YOUNG 프리미엄 실켓 티셔츠,-3.772476


In [20]:
def get_unbuy_clothes(DF,user_id):
    user_rat=DF.loc[user_id,:]
    already_buy=user_rat[user_rat>0].index.tolist()
    clothes_list=DF.columns.tolist()
    unbuy_list=[clothe for clothe in clothes_list if clothe not in already_buy]
    return unbuy_list

In [21]:
unbuy_list=get_unbuy_clothes(user_item_rating,'LV 1 Aqym')
unbuy_list

['#0085 greysh blue crop jeans',
 '#0186 Heritage black mannish crop fit',
 '#0198 Sensitive -A standard fit',
 '#0199 Sensitive -B standard fit',
 '#0203 off-white straight fit',
 '#0214 Moderation indigo straight fit',
 '#0215 Melt black straight fit',
 '#0220 Elaboration straight crop fit',
 '#0223 Melt black straight crop fit',
 '#0245 Black crude wide fit',
 '#3311 데님 점프슈트',
 '(1+1 세트구성) 챔피온 T425 티셔츠',
 '(2PACK) 1301 무지 반팔티 블랙&화이트',
 '(2PACK) K87 포켓 반팔티 15컬러',
 '(2PACK) S600 파워블랜드 무지 맨투맨 크루넥',
 '(2PACK) S700 무지 후드&T425 무지 반팔티',
 '(2PACK) S700 무지 후드티&S600 맨투맨',
 '(2PACK) S700 파워블랜드 풀오버 무지 후드티',
 '(2PACK) T425 무지 반팔티 (11 color)',
 '(2PACK) T425 무지 반팔티 블랙&화이트',
 '(2PACK)S600 맨투맨+S700후드티',
 '(3PACK) 1301 무지 반팔티 블랙&화이트',
 '(BY P.E.DEPT) UNIVERSITY HANDSOME DAN CREWNECK NAVY',
 '(LS-0014)BACK POCKET LEGGINGS INDY PINK',
 '(LS-0014)BACK POCKET LEGGINGS LIGHT GREY',
 '(LS-18557) FOOT HOOK LEGGINGS BLACK',
 '(LS-19166) TIED-UP LEGGINGS BLACK',
 '(LS-19562) PANTS LEGGINGS NAVY',
 '(LS-19563

In [22]:
def recomm_clothes_by_user_id(pred_df,user_id,unbuy_list,top_n):
    recomm_clothes=pred_df.loc[user_id,unbuy_list].sort_values(ascending=False)[:top_n]
    return recomm_clothes

In [23]:
pd.DataFrame(recomm_clothes_by_user_id(user_item_pred_matrix,'LV 1 Aqym',unbuy_list,top_n=10))

,LV 1 Aqym
item_name,
[SET] 클럽 팀 클래식 셋업,4.444396
Arch Logo Hoodie Navy,3.677835
RMTCRW LOGO SWEATSHIRT_BLACK,3.654900
[일주일만이가격] 커츠 B30 워시드,3.615261
(기모) 유니섹스 세미 와이드 밴딩 슬랙스 겨울,3.496557
[패키지] 오버핏 그래픽 반팔티셔츠 3PACK _BB2H18881,3.466792
ARC-Logo Hooded Sweatshirt Black,3.452767
LMC CAPITAL LOGO SWEATSHIRT black,3.418594
KANCO LOGO OXFORD SHIRT beige,3.404435


# 사용자 리뷰 감정 분석

In [25]:
# 클렌징
df['tokens']=df.review.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
idx=df[df.tokens==""].index
a=df.copy()
df=df.drop(idx)
df=df.dropna(subset=['tokens']) # 토큰화 결과 빈값과 null 셀 제거 
df

,item_name,user_id,user_info,score,review,cm,kg,user_size,bmi,tokens
0,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 봄이22222,"XL 구매 남성, 178cm, 66kg",10,가격 저렴한거에 비해 재질은 괜찮습니다배송도 빠르고 이번여름 반팔은 무탠다드 이용할...,178.0,66.0,XL,20.830703,가격 저렴한거에 비해 재질은 괜찮습니다배송도 빠르고 이번여름 반팔은 무탠다드 이용할...
1,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 성진짱123,M 구매,10,무난하게 입을만한거 같습니다 만족합니다람쥐,NaN,NaN,M,NaN,무난하게 입을만한거 같습니다 만족합니다람쥐
2,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 2 뉴비_a0013bf6,"L 구매 남성, 178cm, 90kg",10,사이즈가 딱 좋고 재질이 너무 마음에 드는 재질이에요!! 다른 색상으로 또 구매하려...,178.0,90.0,L,28.405504,사이즈가 딱 좋고 재질이 너무 마음에 드는 재질이에요 다른 색상으로 또 구매하려구요
3,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 뉴비_423608cc,"XL 구매 남성, 178cm, 73kg",10,안에 레이어드 하기도 좋고 두께 적당하고 편해요,178.0,73.0,XL,23.040020,안에 레이어드 하기도 좋고 두께 적당하고 편해요
4,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 쏘쏘_,L 구매,10,진짜 기본티인데 후기 많아서 샀는데 좋아요 깔끔하고 무난해서 좋아요,NaN,NaN,L,NaN,진짜 기본티인데 후기 많아서 샀는데 좋아요 깔끔하고 무난해서 좋아요
...,...,...,...,...,...,...,...,...,...,...
95232,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 선녀와튀었꾼,"블랙 / M , 멜란지 , M 구매 남성, 174cm, 67kg",10,입어보니까 통이랑 기장까지 딱이에요색감도좋아요,174.0,67.0,M,22.129740,입어보니까 통이랑 기장까지 딱이에요색감도좋아요
95233,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 핑플,"멜란지 / L , 블랙 , L 구매 남성, 170cm, 90kg",10,생각보다 크긴한데 재질이나 핏이 나쁘지않습니다.,170.0,90.0,L,31.141869,생각보다 크긴한데 재질이나 핏이 나쁘지않습니다
95234,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 정정휘휘찬찬,"블랙 / M , 멜란지 , M 구매 남성, 170cm, 54kg",10,안조으면 컨버스에 신고 조아서 어글리슈즈에 신음,170.0,54.0,M,18.685121,안조으면 컨버스에 신고 조아서 어글리슈즈에 신음
95235,[패키지] essential 스트링 세미 와이드 팬츠,LV 4 nowgnoj,"블랙 / M , 멜란지 , L 구매 남성, 173cm, 64kg",10,가성비도 좋고 편하게 입기 좋습니다 추천드려요,173.0,64.0,M,21.383942,가성비도 좋고 편하게 입기 좋습니다 추천드려요


In [126]:
tokens=[]
for i in range(df.shape[0]):
    tokens.append(okt.morphs(df['tokens'].iloc[i]))

In [127]:
df['tokenized']=tokens
df

,item_name,user_id,user_info,score,review,cm,kg,user_size,bmi,tokens,tokenized
0,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 봄이22222,"XL 구매 남성, 178cm, 66kg",10,가격 저렴한거에 비해 재질은 괜찮습니다배송도 빠르고 이번여름 반팔은 무탠다드 이용할...,178.0,66.0,XL,20.830703,가격 저렴한거에 비해 재질은 괜찮습니다배송도 빠르고 이번여름 반팔은 무탠다드 이용할...,"[가격, 저렴한거에, 비해, 재질, 은, 괜찮습니다, 배송, 도, 빠르고, 이번, ..."
1,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 성진짱123,M 구매,10,무난하게 입을만한거 같습니다 만족합니다람쥐,NaN,NaN,M,NaN,무난하게 입을만한거 같습니다 만족합니다람쥐,"[무난, 하게, 입을만한, 거, 같습니다, 만족합니, 다람쥐]"
2,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 2 뉴비_a0013bf6,"L 구매 남성, 178cm, 90kg",10,사이즈가 딱 좋고 재질이 너무 마음에 드는 재질이에요!! 다른 색상으로 또 구매하려...,178.0,90.0,L,28.405504,사이즈가 딱 좋고 재질이 너무 마음에 드는 재질이에요 다른 색상으로 또 구매하려구요,"[사이즈, 가, 딱, 좋고, 재질, 이, 너무, 마음, 에, 드는, 재질, 이에요,..."
3,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 뉴비_423608cc,"XL 구매 남성, 178cm, 73kg",10,안에 레이어드 하기도 좋고 두께 적당하고 편해요,178.0,73.0,XL,23.040020,안에 레이어드 하기도 좋고 두께 적당하고 편해요,"[안, 에, 레이어, 드, 하기도, 좋고, 두께, 적당하고, 편해요]"
4,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 쏘쏘_,L 구매,10,진짜 기본티인데 후기 많아서 샀는데 좋아요 깔끔하고 무난해서 좋아요,NaN,NaN,L,NaN,진짜 기본티인데 후기 많아서 샀는데 좋아요 깔끔하고 무난해서 좋아요,"[진짜, 기, 본, 티, 인데, 후기, 많아서, 샀는데, 좋아요, 깔끔하고, 무난,..."
...,...,...,...,...,...,...,...,...,...,...,...
95232,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 선녀와튀었꾼,"블랙 / M , 멜란지 , M 구매 남성, 174cm, 67kg",10,입어보니까 통이랑 기장까지 딱이에요색감도좋아요,174.0,67.0,M,22.129740,입어보니까 통이랑 기장까지 딱이에요색감도좋아요,"[입어보니까, 통, 이랑, 기장, 까지, 딱이에요, 색감, 도, 좋아요]"
95233,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 핑플,"멜란지 / L , 블랙 , L 구매 남성, 170cm, 90kg",10,생각보다 크긴한데 재질이나 핏이 나쁘지않습니다.,170.0,90.0,L,31.141869,생각보다 크긴한데 재질이나 핏이 나쁘지않습니다,"[생각, 보다, 크, 긴한데, 재질, 이나, 핏, 이, 나쁘지, 않습니다]"
95234,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 정정휘휘찬찬,"블랙 / M , 멜란지 , M 구매 남성, 170cm, 54kg",10,안조으면 컨버스에 신고 조아서 어글리슈즈에 신음,170.0,54.0,M,18.685121,안조으면 컨버스에 신고 조아서 어글리슈즈에 신음,"[안조으면, 컨버스, 에, 신고, 조아서, 어글리슈즈, 에, 신음]"
95235,[패키지] essential 스트링 세미 와이드 팬츠,LV 4 nowgnoj,"블랙 / M , 멜란지 , L 구매 남성, 173cm, 64kg",10,가성비도 좋고 편하게 입기 좋습니다 추천드려요,173.0,64.0,M,21.383942,가성비도 좋고 편하게 입기 좋습니다 추천드려요,"[가성, 비도, 좋고, 편하게, 입기, 좋습니다, 추천, 드려요]"


In [32]:
tokens

[['가격',
  '저렴한거에',
  '비해',
  '재질',
  '은',
  '괜찮습니다',
  '배송',
  '도',
  '빠르고',
  '이번',
  '여름',
  '반',
  '팔',
  '은',
  '무탠',
  '다드',
  '이용',
  '할것',
  '같네요'],
 ['무난', '하게', '입을만한', '거', '같습니다', '만족합니', '다람쥐'],
 ['사이즈',
  '가',
  '딱',
  '좋고',
  '재질',
  '이',
  '너무',
  '마음',
  '에',
  '드는',
  '재질',
  '이에요',
  '다른',
  '색상',
  '으로',
  '또',
  '구매',
  '하려구요'],
 ['안', '에', '레이어', '드', '하기도', '좋고', '두께', '적당하고', '편해요'],
 ['진짜',
  '기',
  '본',
  '티',
  '인데',
  '후기',
  '많아서',
  '샀는데',
  '좋아요',
  '깔끔하고',
  '무난',
  '해서',
  '좋아요'],
 ['핏', '이', '너무', '좋습니다', '다른', '색깔', '도', '또', '구입', '할', '예정', '이에요'],
 ['생각',
  '했던',
  '네이비',
  '보다',
  '좀',
  '밝아서',
  '아쉽지만',
  '핏',
  '이',
  '예뻐서',
  '데',
  '일리',
  '로',
  '입기',
  '좋습니다'],
 ['카',
  '고',
  '팬츠',
  '같은거랑',
  '단일',
  '로',
  '매치',
  '했을',
  '때',
  '예쁜거',
  '같아요',
  '목',
  '부분',
  '은',
  '살짝',
  '여유',
  '로운',
  '데',
  '전체',
  '적',
  '으로',
  '빳빳하',
  '게',
  '예쁜핏',
  '이',
  '나옵니다'],
 ['훌륭합니다',
  '조금',
  '목',
  '이',
  '늘어난',
  '것',
  '빼고는요',
  '제',
  '몸',
  '이',

In [128]:
# 불용어 제거
stopwords=['이다','되어다','이라서','라고','하다','아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '는', '은', '수', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로', '바꾸어서 말하면', '바꾸어서 한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에 가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한 까닭에', '하기 때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느곳', '더군다나', '하물며', '더욱이는', '어느때', '언제', '야', '이봐', '어이', '여보시오', '흐흐', '흥', '휴', '헉헉', '헐떡헐떡', '영차', '여차', '어기여차', '끙끙', '아야', '앗', '아야', '콸콸', '졸졸', '좍좍', '뚝뚝', '주룩주룩', '솨', '우르르', '그래도', '또', '그리고', '바꾸어말하면', '바꾸어말하자면', '혹은', '혹시', '답다', '및', '그에 따르는', '때가 되어', '즉', '지든지', '설령', '가령', '하더라도', '할지라도', '일지라도', '지든지', '몇', '거의', '하마터면', '인젠', '이젠', '된바에야', '된이상', '만큼\t어찌됏든', '그위에', '게다가', '점에서 보아', '비추어 보아', '고려하면', '하게될것이다', '일것이다', '비교적', '좀', '보다더', '비하면', '시키다', '하게하다', '할만하다', '의해서', '연이서', '이어서', '잇따라', '뒤따라', '뒤이어', '결국', '의지하여', '기대여', '통하여', '자마자', '더욱더', '불구하고', '얼마든지', '마음대로', '주저하지 않고', '곧', '즉시', '바로', '당장', '하자마자', '밖에 안된다', '하면된다', '그래', '그렇지', '요컨대', '다시 말하자면', '바꿔 말하면', '즉', '구체적으로', '말하자면', '시작하여', '시초에', '이상', '허', '헉', '허걱', '바와같이', '해도좋다', '해도된다', '게다가', '더구나', '하물며', '와르르', '팍', '퍽', '펄렁', '동안', '이래', '하고있었다', '이었다', '에서', '로부터', '까지', '예하면', '했어요', '해요', '함께', '같이', '더불어', '마저', '마저도', '양자', '모두', '습니다', '가까스로', '하려고하다', '즈음하여', '다른', '다른 방면으로', '해봐요', '습니까', '했어요', '말할것도 없고', '무릎쓰고', '개의치않고', '하는것만 못하다', '하는것이 낫다', '매', '매번', '들', '모', '어느것', '어느', '로써', '갖고말하자면', '어디', '어느쪽', '어느것', '어느해', '어느 년도', '라 해도', '언젠가', '어떤것', '어느것', '저기', '저쪽', '저것', '그때', '그럼', '그러면', '요만한걸', '그래', '그때', '저것만큼', '그저', '이르기까지', '할 줄 안다', '할 힘이 있다', '너', '너희', '당신', '어찌', '설마', '차라리', '할지언정', '할지라도', '할망정', '할지언정', '구토하다', '게우다', '토하다', '메쓰겁다', '옆사람', '퉤', '쳇', '의거하여', '근거하여', '의해', '따라', '힘입어', '그', '다음', '버금', '두번째로', '기타', '첫번째로', '나머지는', '그중에서', '견지에서', '형식으로 쓰여', '입장에서', '위해서', '단지', '의해되다', '하도록시키다', '뿐만아니라', '반대로', '전후', '전자', '앞의것', '잠시', '잠깐', '하면서', '그렇지만', '다음에', '그러한즉', '그런즉', '남들', '아무거나', '어찌하든지', '같다', '비슷하다', '예컨대', '이럴정도로', '어떻게', '만약', '만일', '위에서 서술한바와같이', '인 듯하다', '하지 않는다면', '만약에', '무엇', '무슨', '어느', '어떤', '아래윗', '조차', '한데', '그럼에도 불구하고', '여전히', '심지어', '까지도', '조차도', '하지 않도록', '않기 위하여', '때', '시각', '무렵', '시간', '동안', '어때', '어떠한', '하여금', '네', '예', '우선', '누구', '누가 알겠는가', '아무도', '줄은모른다', '줄은 몰랏다', '하는 김에', '겸사겸사', '하는바', '그런 까닭에', '한 이유는', '그러니', '그러니까', '때문에', '그', '너희', '그들', '너희들', '타인', '것', '것들', '너', '위하여', '공동으로', '동시에', '하기 위하여', '어찌하여', '무엇때문에', '붕붕', '윙윙', '나', '우리', '엉엉', '휘익', '윙윙', '오호', '아하', '어쨋든', '만 못하다', '하기보다는', '차라리', '하는 편이 낫다', '흐흐', '놀라다', '상대적으로 말하자면', '마치', '아니라면', '쉿', '그렇지 않으면', '그렇지 않다면', '안 그러면', '아니었다면', '하든지', '아니면', '이라면', '좋아', '알았어', '하는것도', '그만이다', '어쩔수 없다', '하나', '일', '일반적으로', '일단', '한켠으로는', '오자마자', '이렇게되면', '이와같다면', '전부', '한마디', '한항목', '근거로', '하기에', '아울러', '하지 않도록', '않기 위해서', '이르기까지', '이 되다', '로 인하여', '까닭으로', '이유만으로', '이로 인하여', '그래서', '이 때문에', '그러므로', '그런 까닭에', '알 수 있다', '결론을 낼 수 있다', '으로 인하여', '있다', '어떤것', '관계가 있다', '관련이 있다', '연관되다', '어떤것들', '에 대해', '이리하여', '그리하여', '여부', '하기보다는', '하느니', '하면 할수록', '운운', '이러이러하다', '하구나', '하도다', '다시말하면', '다음으로', '에 있다', '에 달려 있다', '우리', '우리들', '오히려', '하기는한데', '어떻게', '어떻해', '어찌됏어', '어때', '어째서', '본대로', '자', '이', '이쪽', '여기', '이것', '이번', '이렇게말하자면', '이런', '이러한', '이와 같은', '요만큼', '요만한 것', '얼마 안 되는 것', '이만큼', '이 정도의', '이렇게 많은 것', '이와 같다', '이때', '이렇구나', '것과 같이', '끼익', '삐걱', '따위', '와 같은 사람들', '부류의 사람들', '왜냐하면', '중의하나', '오직', '오로지', '에 한하다', '하기만 하면', '도착하다', '까지 미치다', '도달하다', '정도에 이르다', '할 지경이다', '결과에 이르다', '관해서는', '여러분', '하고 있다', '한 후', '혼자', '자기', '자기집', '자신', '우에 종합한것과같이', '총적으로 보면', '총적으로 말하면', '총적으로', '대로 하다', '으로서', '참', '그만이다', '할 따름이다', '쿵', '탕탕', '쾅쾅', '둥둥', '봐', '봐라', '아이야', '아니', '와아', '응', '아이', '참나', '년', '월', '일', '령', '영', '일', '이', '삼', '사', '오', '육', '륙', '칠', '팔', '구', '이천육', '이천칠', '이천팔', '이천구', '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '령', '영']
clean_words=[]
for sentence in tokens:
    x=[word for word in sentence if not word in stopwords]
    clean_words.append(x)
clean_words

[['가격',
  '저렴한거에',
  '비해',
  '재질',
  '괜찮습니다',
  '배송',
  '도',
  '빠르고',
  '여름',
  '반',
  '무탠',
  '다드',
  '이용',
  '할것',
  '같네요'],
 ['무난', '하게', '입을만한', '거', '같습니다', '만족합니', '다람쥐'],
 ['사이즈', '좋고', '재질', '너무', '마음', '드는', '재질', '이에요', '색상', '구매', '하려구요'],
 ['안', '레이어', '드', '하기도', '좋고', '두께', '적당하고', '편해요'],
 ['진짜',
  '기',
  '본',
  '티',
  '인데',
  '후기',
  '많아서',
  '샀는데',
  '좋아요',
  '깔끔하고',
  '무난',
  '해서',
  '좋아요'],
 ['핏', '너무', '좋습니다', '색깔', '도', '구입', '할', '예정', '이에요'],
 ['생각', '했던', '네이비', '보다', '밝아서', '아쉽지만', '핏', '예뻐서', '데', '일리', '입기', '좋습니다'],
 ['카',
  '고',
  '팬츠',
  '같은거랑',
  '단일',
  '매치',
  '했을',
  '예쁜거',
  '같아요',
  '목',
  '부분',
  '살짝',
  '여유',
  '로운',
  '데',
  '전체',
  '적',
  '빳빳하',
  '게',
  '예쁜핏',
  '나옵니다'],
 ['훌륭합니다', '목', '늘어난', '빼고는요', '몸', '문제', '겠죠', '수고', '하십', '쇼'],
 ['질', '좋고', '얇아서', '좋아요', '생각', '입니다', '소매', '길어오'],
 ['무지', '반', '팔이', '필요했었는데', '역시', '무탠다드', '최고', '에요'],
 ['기본',
  '반',
  '티러',
  '매우',
  '만족하고',
  '레이어',
  '드용',
  '구매',
  '했지만',
  '단',
  '품',
  '으로도',
  '착용

In [34]:
import csv

In [129]:
# 감성사전 read 하기 
table=dict()

with open('polarity.csv','r',-1,'utf-8') as polarity:
    
    reader=csv.reader(polarity) # csv.reader형식의 객체를 반환
    next(polarity) # 다음행으로 넘어가라
    
    for line in reader:
        key=str()
        for word in line[0].split(';'):
            key+=word.split('/')[0]
            
        table[key]={'Neg':line[3],"Nuet":line[4],'Pos':line[6]} # 이중 딕셔너리

In [130]:
table

{'가*': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가*있': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가*있었': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가*ㄴ다*': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가': {'Neg': '0.333333333', 'Nuet': '0', 'Pos': '0.555555556'},
 '가되': {'Neg': '0.363636364', 'Nuet': '0.272727273', 'Pos': '0.363636364'},
 '가되ㄴ': {'Neg': '0', 'Nuet': '1', 'Pos': '0'},
 '가되ㄹ': {'Neg': '0', 'Nuet': '1', 'Pos': '0'},
 '가되어': {'Neg': '1', 'Nuet': '0', 'Pos': '0'},
 '가되어야지요': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가아니': {'Neg': '0.666666667', 'Nuet': '0.166666667', 'Pos': '0.166666667'},
 '가아니ㄴ가': {'Neg': '1', 'Nuet': '0', 'Pos': '0'},
 '가아니면': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가아니ㅂ니다': {'Neg': '1', 'Nuet': '0', 'Pos': '0'},
 '가가능': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가가능하': {'Neg': '0', 'Nuet': '0', 'Pos': '1'},
 '가가장': {'Neg': '1', 'Nuet': '0', 'Pos': '0'},
 '가가장많이': {'Neg': '1', 'Nuet': '0', 'Pos': '0'},
 '가가장어렵': {'Neg': '1', 'Nuet': '0', 'Pos': '0'},
 '가굵': {'Neg':

In [134]:
# 감성분석 점수를 담아둘 빈 데이터 프레임 만들기
columns=['negative','neutral','positive']
sent=pd.DataFrame(columns=columns)
table['가']['Neg']

'0.333333333'

In [135]:
# 불용어 처리까지 한 단어 리스트에 감성 분석 점수 산출
clean_words # 토큰화 후 불용어처리 된 단어 리스트

negative_list=[]
neutral_list=[]
positive_list=[]


for words in clean_words:
    
    negative=0
    neutral=0
    positive=0
    
    for word in words:
        if word in table:
            negative+=float(table[word]['Neg'])
            neutral+=float(table[word]['Nuet'])
            positive+=float(table[word]['Pos'])
    
    negative_list.append(negative)
    neutral_list.append(neutral)
    positive_list.append(positive)
    

In [137]:
len(positive_list)

95222

In [138]:
sent['negative']=negative_list
sent['positive']=positive_list
sent['neutral']=neutral_list

In [139]:
result=[]
for i in range(sent.shape[0]):
    val=sent.positive.iloc[i]/(sent.negative.iloc[i]+sent.positive.iloc[i])
    if val > 0.52 :
        result.append(val)
    elif val < 0.48:
        result.append(0)
    else:
        result.append(-val)

In [140]:
result

[0.75,
 0,
 0.5217391304097879,
 -0.5,
 0,
 0.6345177665212709,
 0.7894736842105263,
 0.8586222778265172,
 0,
 0,
 0.5238095237891156,
 0.6666666666666666,
 0.6256247160950136,
 0.7666157372817116,
 1.0,
 0.5396825397142857,
 0.6666666666666666,
 0.5354838709852238,
 0.6666666666666666,
 0.6896551723923899,
 0,
 0.6357142857012755,
 0,
 0.5555555556131687,
 0.8333333333333334,
 0.5300546449354117,
 0.666666667,
 0.9131545337931035,
 0.7192982457447262,
 0,
 nan,
 0,
 0.846153846260355,
 0.8961038961969978,
 1.0,
 0.9666666666666667,
 0,
 -0.5058823528719031,
 0.7600000000192001,
 0.75609756102558,
 0.6356347439066949,
 0.7480577135387811,
 0.71250000007125,
 0.5974842768220694,
 0,
 0,
 1.0,
 0.5434782608745274,
 0.8445595855463501,
 0,
 0,
 0.6098981077054023,
 0.6666666662777778,
 0.7659259259918024,
 0.8589743589891518,
 0.8333333333333334,
 nan,
 0,
 0.83333333325,
 0.8333333333333334,
 1.0,
 0,
 0,
 0.8888888889629629,
 1.0,
 1.0,
 0.7857142857295919,
 0,
 -0.5,
 0.664854493209530

In [141]:
sent['result']=result

In [142]:
df.reset_index(inplace=True)

In [34]:
sent=pd.read_excel(r'C:\Users\PC\프로젝트\sent.xlsx')

In [35]:
sent_df=pd.concat([df,sent],axis=1)

In [38]:
sent_df

,item_name,user_id,user_info,score,review,cm,kg,user_size,bmi,tokens,negative,neutral,positive,result
0,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 봄이22222,"XL 구매 남성, 178cm, 66kg",10.0,가격 저렴한거에 비해 재질은 괜찮습니다배송도 빠르고 이번여름 반팔은 무탠다드 이용할...,178.0,66.0,XL,20.830703,가격 저렴한거에 비해 재질은 괜찮습니다배송도 빠르고 이번여름 반팔은 무탠다드 이용할...,1.000000,0.000000,3.000000,0.750000
1,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 성진짱123,M 구매,10.0,무난하게 입을만한거 같습니다 만족합니다람쥐,NaN,NaN,M,NaN,무난하게 입을만한거 같습니다 만족합니다람쥐,1.750000,0.000000,1.000000,0.000000
2,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 2 뉴비_a0013bf6,"L 구매 남성, 178cm, 90kg",10.0,사이즈가 딱 좋고 재질이 너무 마음에 드는 재질이에요!! 다른 색상으로 또 구매하려...,178.0,90.0,L,28.405504,사이즈가 딱 좋고 재질이 너무 마음에 드는 재질이에요 다른 색상으로 또 구매하려구요,0.831933,0.058824,0.907563,0.521739
3,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 뉴비_423608cc,"XL 구매 남성, 178cm, 73kg",10.0,안에 레이어드 하기도 좋고 두께 적당하고 편해요,178.0,73.0,XL,23.040020,안에 레이어드 하기도 좋고 두께 적당하고 편해요,0.500000,0.000000,0.500000,-0.500000
4,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 쏘쏘_,L 구매,10.0,진짜 기본티인데 후기 많아서 샀는데 좋아요 깔끔하고 무난해서 좋아요,NaN,NaN,L,NaN,진짜 기본티인데 후기 많아서 샀는데 좋아요 깔끔하고 무난해서 좋아요,2.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95232,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 선녀와튀었꾼,"블랙 / M , 멜란지 , M 구매 남성, 174cm, 67kg",10.0,입어보니까 통이랑 기장까지 딱이에요색감도좋아요,174.0,67.0,M,22.129740,입어보니까 통이랑 기장까지 딱이에요색감도좋아요,NaN,NaN,NaN,NaN
95233,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 핑플,"멜란지 / L , 블랙 , L 구매 남성, 170cm, 90kg",10.0,생각보다 크긴한데 재질이나 핏이 나쁘지않습니다.,170.0,90.0,L,31.141869,생각보다 크긴한데 재질이나 핏이 나쁘지않습니다,NaN,NaN,NaN,NaN
95234,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 정정휘휘찬찬,"블랙 / M , 멜란지 , M 구매 남성, 170cm, 54kg",10.0,안조으면 컨버스에 신고 조아서 어글리슈즈에 신음,170.0,54.0,M,18.685121,안조으면 컨버스에 신고 조아서 어글리슈즈에 신음,NaN,NaN,NaN,NaN
95235,[패키지] essential 스트링 세미 와이드 팬츠,LV 4 nowgnoj,"블랙 / M , 멜란지 , L 구매 남성, 173cm, 64kg",10.0,가성비도 좋고 편하게 입기 좋습니다 추천드려요,173.0,64.0,M,21.383942,가성비도 좋고 편하게 입기 좋습니다 추천드려요,NaN,NaN,NaN,NaN


In [39]:
sen_df=sent_df.drop(['tokens','negative','neutral','positive'],axis=1)

In [40]:
sen_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95237 entries, 0 to 95236
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item_name  95222 non-null  object 
 1   user_id    95221 non-null  object 
 2   user_info  94970 non-null  object 
 3   score      95222 non-null  float64
 4   review     95222 non-null  object 
 5   cm         67602 non-null  float64
 6   kg         67587 non-null  float64
 7   user_size  82277 non-null  object 
 8   bmi        67584 non-null  float64
 9   result     92758 non-null  float64
dtypes: float64(5), object(5)
memory usage: 8.0+ MB


In [41]:
# 새로운 점수 적용 후 다시 추천시스템 돌려보기
sen_df['new_score']=sen_df.score+sen_df.result

In [42]:
new_DF=pd.pivot_table(sen_df,index='user_id',columns='item_name',values='new_score')
P_n,Q_n=matrix_factorization(new_DF.values,K=50,steps=200,learning_rate=0.01,r_lambda=0.01)
new_pred_matrix=np.dot(P_n,Q_n.T)
new_ratings_matrix=pd.DataFrame(data=new_pred_matrix,index=new_DF.index,columns=new_DF.columns)
new_ratings_matrix

### iteration step:  0 rmse: 10.059450667412307
### iteration step:  10 rmse: 1.0072668160178664
### iteration step:  20 rmse: 0.5117413017001577
### iteration step:  30 rmse: 0.2850059104299373
### iteration step:  40 rmse: 0.11150218936081631
### iteration step:  50 rmse: 0.053250069159899785
### iteration step:  60 rmse: 0.03697921101761055
### iteration step:  70 rmse: 0.032387658248056936
### iteration step:  80 rmse: 0.03105354824808933
### iteration step:  90 rmse: 0.03054240093878416
### iteration step:  100 rmse: 0.030243013000767798
### iteration step:  110 rmse: 0.029978013907141304
### iteration step:  120 rmse: 0.02970152721392082
### iteration step:  130 rmse: 0.029408288538879425
### iteration step:  140 rmse: 0.029102078552671957
### iteration step:  150 rmse: 0.02878750124844838
### iteration step:  160 rmse: 0.028468438562525438
### iteration step:  170 rmse: 0.028147947577101668
### iteration step:  180 rmse: 0.02782839153040175
### iteration step:  190 rmse: 0.02751

item_name,#0085 greysh blue crop jeans,#0186 Heritage black mannish crop fit,#0198 Sensitive -A standard fit,#0199 Sensitive -B standard fit,#0203 off-white straight fit,#0214 Moderation indigo straight fit,#0215 Melt black straight fit,#0220 Elaboration straight crop fit,#0223 Melt black straight crop fit,#0245 Black crude wide fit,...,헤비웨이트 오버사이즈 후디드 스웨트셔츠 [미디엄 그레이],헤비코튼 자수로고 나시 3 color,화란 세미오버 니트 라벤더,화란 세미오버 니트 블랙,화이트 중기장 레이어드 슬리브리스 티셔츠 (421142LY61),화이트라벨 오버롤 네이비,후디드 스웨트셔츠 [멜란지 그레이],휩코드 세미 와이드 히든 밴딩 슬랙스 [블랙],휩코드 와이드 히든 밴딩 슬랙스 [블랙],히든 버튼 드레스 셔츠 [화이트]
user_id,,,,,,,,,,,,,,,,,,,,,
LV 1 Aqym,1.712893,-0.144622,1.705714,-1.527127,0.519900,-1.160464,1.796001,0.730976,-0.718759,0.196305,...,1.635950,0.614899,0.819753,0.074934,-0.904150,-1.219058,-0.152238,1.103998,-0.058995,4.077009
LV 1 Ehrlich,-0.066288,0.463653,1.465956,-0.436705,0.741084,2.034974,0.813560,0.264927,-1.475094,-0.243439,...,-0.642337,0.377774,1.523322,0.403128,0.646702,1.691246,1.608754,0.473310,0.933636,0.096453
LV 1 Yang30,-0.457000,-2.663970,0.533845,-0.340147,0.154971,0.873975,-1.110427,1.318831,-2.210762,-0.233224,...,-2.262906,-0.611523,-0.077232,-0.938641,-0.754317,-1.317252,-1.535886,0.536671,-2.735418,-0.376701
LV 1 Yuha810,-1.322906,-1.189036,-0.185087,0.951600,-1.258641,-0.351313,-0.462932,2.244722,1.996531,-0.734814,...,-1.011946,-0.588214,1.619131,-0.926533,1.764523,-0.227881,1.548146,0.003210,2.370029,0.947805
LV 1 filter7,-1.038872,1.829924,0.246635,-3.043367,-1.254787,-2.193628,0.956588,3.401930,1.526392,-1.007727,...,0.033302,0.625409,1.500004,0.658224,0.640651,-1.605375,2.123381,-1.199113,0.716935,-0.379884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LV 8 훈형,-1.223062,2.994972,10.781135,-0.252199,-1.183267,0.290534,6.586897,11.403246,0.360444,3.666782,...,-5.056482,-1.287238,5.505589,2.966297,-1.891409,6.499121,0.493002,5.474160,-0.676299,-2.847810
LV 8 휘휘꾸,-0.603920,0.944625,0.280028,-1.655594,-0.355100,1.523908,0.234180,0.951098,-0.108355,0.899479,...,-1.151241,-0.326450,0.224940,-2.396783,2.527599,2.546723,-1.809412,-1.028943,-1.062518,1.278682
LV 8 ｎｏ,0.753187,1.432925,1.201192,2.152370,0.987151,1.801058,-1.458144,1.989934,-0.236276,-1.325226,...,-2.008449,-2.785146,1.663258,1.013392,1.123190,1.640013,-1.642156,2.712405,-0.117126,0.129457


In [148]:
new_unbuy_list=get_unbuy_clothes(new_DF,'LV 1 Aqym')
pd.DataFrame(recomm_clothes_by_user_id(new_ratings_matrix,'LV 1 Aqym',new_unbuy_list,top_n=10))


,LV 1 Aqym
item_name,
[UNISEX] 레이어드 오버핏 슬리브리스 나시 (화이트),4.528768
페이스 로고 엠브로이더드 후디 (네이비),4.455900
워시드 린넨 와이드 팬츠(블랙),4.455748
남성용 프로 타이츠 BV5641_010,4.359780
블록 후드 (2 컬러),4.179631
[일주일만이가격] 1981 BLACK STANDARD3 JEANS [CROP SLIM],3.942697
Off White / Ordinary Crop,3.930113
[Mmlg] 1987MMLG HF-T (ULTRA MARINE),3.915790
MIXED BOUCLE KNIT LONG SLEEVE BLACK,3.914178


In [149]:
pd.DataFrame(recomm_clothes_by_user_id(user_item_pred_matrix,'LV 1 Aqym',unbuy_list,top_n=10))

,LV 1 Aqym
item_name,
[SET] 클럽 팀 클래식 셋업,4.444396
Arch Logo Hoodie Navy,3.677835
RMTCRW LOGO SWEATSHIRT_BLACK,3.654900
[일주일만이가격] 커츠 B30 워시드,3.615261
(기모) 유니섹스 세미 와이드 밴딩 슬랙스 겨울,3.496557
[패키지] 오버핏 그래픽 반팔티셔츠 3PACK _BB2H18881,3.466792
ARC-Logo Hooded Sweatshirt Black,3.452767
LMC CAPITAL LOGO SWEATSHIRT black,3.418594
KANCO LOGO OXFORD SHIRT beige,3.404435


In [153]:
user_item_rating

item_name,#0085 greysh blue crop jeans,#0186 Heritage black mannish crop fit,#0198 Sensitive -A standard fit,#0199 Sensitive -B standard fit,#0203 off-white straight fit,#0214 Moderation indigo straight fit,#0215 Melt black straight fit,#0220 Elaboration straight crop fit,#0223 Melt black straight crop fit,#0245 Black crude wide fit,...,헤비웨이트 오버사이즈 후디드 스웨트셔츠 [미디엄 그레이],헤비코튼 자수로고 나시 3 color,화란 세미오버 니트 라벤더,화란 세미오버 니트 블랙,화이트 중기장 레이어드 슬리브리스 티셔츠 (421142LY61),화이트라벨 오버롤 네이비,후디드 스웨트셔츠 [멜란지 그레이],휩코드 세미 와이드 히든 밴딩 슬랙스 [블랙],휩코드 와이드 히든 밴딩 슬랙스 [블랙],히든 버튼 드레스 셔츠 [화이트]
user_id,,,,,,,,,,,,,,,,,,,,,
LV 1 Aqym,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 Ehrlich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 Yang30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 Yuha810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 filter7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LV 8 훈형,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 8 휘휘꾸,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 8 ｎｏ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings(action='ignore')
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances

In [ ]:
lower1=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\상의1.xlsx',thousands = ',')
lower2=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\상의2.xlsx',thousands = ',')
lower3=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\하의1.xlsx',thousands = ',')
lower4=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\하의2.xlsx',thousands = ',')

def remove(a):
    a.price=a.price.str.replace('\n','')
    a.tag=a.tag.str.replace('\n','')
    a=a.replace('Null',np.nan)
    return a

lower1=remove(lower1)
lower2=remove(lower2)
lower3=remove(lower3)
lower4=remove(lower4)

# 모든 컨텐츠데이터 불러온 후 하나의 데이터프레임으로 만들기
df=pd.concat([upper1,upper2,lower1,lower2]).reset_index()

# 태그 null값인거 제거 
DF=df[df['tag'].notnull()]
DF.reset_index(inplace=True)
DF.drop(['level_0','index'],axis=1,inplace=True)

# 태그를 # 기준으로 나누기
DF.tag=DF.tag.str.split('#')

#앞의 , 없애주기
for i in range(DF.shape[0]):
    del DF.tag[i][0]

# item_name에 붙어있는 색상 일부분제거 - 추후 개선 필요
DF['item_name']=DF.item_name.str.replace(r'\[[^)]*\]', '')
DF['item_name']=DF.item_name.str.replace(r'\([^)]*\)', '')

# 중복되는 item_name 제거 
DF=DF.drop_duplicates(['item_name'])
DF.reset_index(inplace=True)
DF.drop('index',axis=1,inplace=True)

# 수치형 변수 데이터 타입 변경
DF=DF.astype({'score_average':'float'})
DF=DF.astype({'review_count':'float'})
DF.describe()

In [ ]:
# 가중평균함수 
# 리뷰개수와 평균평점으로 가중치 적용
# 리뷰 개수가 적고 평균평점이 높을 경우 평점이 높이 반영되는 경우를 막기 위해
def weighted_vote_average(record):
    r=record['review_count']
    R=record['score_average'] 
    return ( (r/(r+m)*R) ) + ( (m/(m+r)*M)  )

M=np.round(DF['score_average'].mean(),4) # 전체 평점 평균
m=DF['review_count'].quantile(0.5)       # 리뷰 조회수 50%


DF['weighted_vote']=DF.apply(weighted_vote_average,axis=1) # 함수적용해서 열방향으로 붙혀주기

DF[['item_name','score_average','review_count','weighted_vote']].sort_values( by='weighted_vote',ascending=False)

# TFIDF
DF.tag_tfidf=DF.tag.apply(lambda x : (' ').join(x))
DF['tag_tfidf']=DF.tag_tfidf

tfidf=TfidfVectorizer(min_df=1,ngram_range=(1,3))
tag_tf=tfidf.fit_transform(DF['tag_tfidf'])

# 1. 코사인유사도
tf_sim=cosine_similarity(tag_tf,tag_tf)



# Word2vec 
word=[]
for words in DF.tag:
    word.append(words)

word2vec=Word2Vec(sentences=word,window=5,sg=1,min_count=1,vector_size=100,epochs=100)


# 단어 벡터 평균구하기
def vectors(embedding):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(word2vec.wv[word]) 
            count+=1
        except : continue
    
    
    word_embedding2=np.array(word_embedding)
    
    
    v=word_embedding2.sum(axis=0)  
    
    
    if type(v)!=np.ndarray:
        return np.zeors(100)
    
    return(v/count)



wordvec=[vectors(x) for x in word]

# 1. 코사인유사도
w2v_sim=cosine_similarity(wordvec,wordvec)


#최종 알고리즘
def sim_clothes(clothe,sim,top_n):
    
    # 찾고자 하는 옷선택
    clothes=DF[DF.item_name==clothe]
    # 찾고자 하는 옷의 인덱스 값만 추출하여 새로운 변수에 저장
    clothes_index=clothes.index.values
    
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]
    
    # 내가 추출하려는 옷의 인덱스의 행번째추출, 이중리스트이므로 [0][0]
    # -> 아이템이름을 추출 -> 내가 원하는 옷의 이름과 같지 않다면
    if DF.iloc[sim_sorted_ind[clothes.index][0][0]].item_name != clothe:
        
        # 내가 추출하려는 옷의 인덱스 행번째의 array를 리스트로 변경, 변수명 l로 저장
        l=sim_sorted_ind[clothes.index][0].tolist()
        # 내가 추출하려는 옷의 인덱스 값 자체를 리스트로 변경한 후 값으로 추출
        v=clothes.index.values.tolist()[0]
        
        # 리스트 l 에서 내가 추출하려고 한 옷의 값을 제거
        # -> 내가 추출하고자 하는 옷의 인덱스 번호가 제거된 리스트 생성
        l.remove(clothes.index.values.tolist()[0])
        
        # 리스트l의 array형태로 맞춰주기
        sim_sorted_ind2=np.array(l)
        
    
        # 자기 자신을 제외하고 유사도가 높은 인덱스를 top_n개수 만큼 추출
        # 내가 추출하고자 하는 인덱스의 값이 제거된 상태이므로 0값부터 원하는 개수만큼 출력
        similar_indexes=sim_sorted_ind2[0:(top_n)]
        a=  DF.iloc[similar_indexes].sort_values(by='weighted_vote',ascending=False).drop('tag_tfidf',axis=1)
    
    else :
        # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
        similar_indexes=sim_sorted_ind[clothes_index,1:(top_n+1)]
        a= DF.iloc[similar_indexes[0]].sort_values(by='weighted_vote',ascending=False).drop('tag_tfidf',axis=1)
    
    return a